In [1]:
import calendar
import xmlrpc.client
import csv
import pandas as pd
import phonenumbers
import psycopg2

ODOO_URL = "https://smsperkasa.odoo.com"
ODOO_DB = "smsperkasa-master-1574977"
ODOO_USERNAME = "admin"
ODOO_PASSWORD = "q82fD^YMXL246cUNr2Qp{a{TN"


In [2]:
df = None

In [3]:
def search_from_odoo(model, payload, fields, order=''):
    """Mixin function to search data from Odoo with Odoo's xmlrpc"""
    try:
        common = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/common')
        dbname = ODOO_DB
        user = ODOO_USERNAME
        pw = ODOO_PASSWORD

        uid = common.authenticate(dbname, user, pw, {})

        models = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/object')
        
        result = models.execute_kw(
            dbname,uid,pw,
            model,'search_read',[payload],
            {
                'fields': fields, 
                'context': {'lang': 'en_GB'}, 
                'order': order
            }
        )

        return result
    except Exception as err:
        return ["Error", err]

# def convert_phonenumber(phonenumber):
#     """Function to convert phone number to the right format"""
#     if pd.isna(phonenumber) == False:
#         phone = phonenumber
#         res = ''
#         try:
#             res = phonenumbers.format_number(phonenumbers.parse(phone.replace("+69","+62"),"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
#         except:
#             # print(phone)
#             res = phonenumbers.format_number(phonenumbers.parse(phone.replace("+","+62"),"ID"), phonenumbers.PhoneNumberFormat.NATIONAL).replace('0', '+62 ',1)
#         return res
#     else:
#         return None

def convert_phonenumber(phonenumber):
    """Function to convert phone number to the right format"""
    try:
        if phonenumber[0] == '0':
            phonenumber = '+62' + phonenumber[1:]
        parsed_phone = phonenumbers.parse(phonenumber, 'ID')
        return phonenumbers.format_number(
            parsed_phone,
            phonenumbers.PhoneNumberFormat.INTERNATIONAL
        )
    except Exception as e:
        if phonenumber is None:
            return None
        return "error"

def reverse_param(feature_param):
    if feature_param == 'cleaned_email':
        return 'cleaned_phone'
    return 'cleaned_email'

def duplicated_label(feature_param, row, depth, label):
    if depth >= 4:
        return None
    found_dup = df[(df[feature_param] == row[feature_param])]
    if len(found_dup) > 1:
        for i, r in found_dup.loc[:, [reverse_param(feature_param)]].iterrows():
            df.loc[(df[reverse_param(feature_param)] == r[reverse_param(feature_param)]), 'group_label'] = label
            duplicated_label(reverse_param(feature_param), r, depth + 1, label)
        df.loc[(df[feature_param] == row[feature_param]), 'group_label'] = label
    return None


In [10]:
conn = psycopg2.connect(
    host="167.71.206.23",
    database="chatwoot_production",
    user="postgres",
    password="s0f!a_s3xy?!",
)

cursor = conn.cursor()
query = """
SELECT * FROM contacts
WHERE
    (email IS NOT NULL AND email <> '') OR
    (phone_number IS NOT NULL AND phone_number <> '')
ORDER BY id
"""
cursor.execute(query)

records = cursor.fetchall()
cursor.close()

raw_cw_df = pd.DataFrame(records, columns=['id', 'name', 'email', 'phone_number', 'account_id', 'created_at', 'updated_at', 'additional_attributes', 'identifier', 'custom_attributes', 'last_activity_at'])
cw_df = raw_cw_df
cw_df.rename(columns={
    'created_at': 'create_date',
    'phone_number': 'phone',
    'email': 'cleaned_email'
}, inplace=True)
cw_df = cw_df.replace('', None)
cw_df['create_date'] = pd.to_datetime(cw_df['create_date'], errors='coerce')
cw_df['cleaned_phone'] = cw_df.apply(lambda x: convert_phonenumber(x.phone), axis=1)
cw_df = cw_df[cw_df.cleaned_phone != 'error']

In [12]:
odoo_contacts = search_from_odoo(
    'res.partner',
    [
        ['email', 'not like', 'smsperkasa'],
    ],
    ['id', 'name', 'email', 'phone', 'mobile', 'create_date']
)
raw_odoo_contacts_df = pd.DataFrame(odoo_contacts)

odoo_contacts_df = raw_odoo_contacts_df.replace('', None)
odoo_contacts_df = odoo_contacts_df.replace(False, None)
odoo_contacts_df['create_date'] = pd.to_datetime(odoo_contacts_df['create_date'], errors='coerce')

odoo_contacts_df[['phone1', 'phone2', 'phone3']] = odoo_contacts_df['phone'].str.split(r'[,/<>;]+', expand=True)
odoo_contacts_df = odoo_contacts_df.melt(id_vars=['id', 'name', 'email', 'phone', 'mobile', 'create_date'], value_vars=['phone1', 'phone2', 'phone3'], value_name='cleaned_phone')
odoo_contacts_df = odoo_contacts_df[~(((odoo_contacts_df.cleaned_phone.isna()) | (odoo_contacts_df.cleaned_phone.str.strip() == "")) & ((odoo_contacts_df.variable == 'phone2') | (odoo_contacts_df.variable == 'phone3')))]
odoo_contacts_df.drop(columns='variable', inplace=True)

tmp_contact_copy = odoo_contacts_df
work_phones = tmp_contact_copy[['id', 'name', 'email', 'phone', 'create_date', 'cleaned_phone']]
other_phones = (
    tmp_contact_copy[~tmp_contact_copy.mobile.isna()]['mobile'].str.split(',')
    .explode()
    .to_frame('cleaned_phone')
    
    # grab the correct name from the original DataFrame
    .join(tmp_contact_copy[['id', 'name', 'email', 'phone', 'create_date']])
)

tmp_combined_contact = pd.concat([work_phones, other_phones]).reset_index(level=0)
sorted_contacts_df = tmp_combined_contact.sort_values('create_date', ascending=False)
sorted_contacts_df = sorted_contacts_df.drop_duplicates(subset=['id', 'name', 'email', 'phone', 'create_date', 'cleaned_phone'], keep='last')
sorted_contacts_df = sorted_contacts_df[['id', 'name', 'email', 'phone', 'create_date', 'cleaned_phone']]

sorted_contacts_df[['email1', 'email2', 'email3']] = sorted_contacts_df['email'].str.split(r'[ ,/<>]+', expand=True)
sorted_contacts_df = sorted_contacts_df.melt(id_vars=['id', 'name', 'email', 'phone', 'create_date', 'cleaned_phone'], value_vars=['email1', 'email2', 'email3'], value_name='cleaned_email')
sorted_contacts_df = sorted_contacts_df[~(((sorted_contacts_df.cleaned_email.isna()) | (sorted_contacts_df.cleaned_email.str.strip() == "")) & ((sorted_contacts_df.variable == 'email2') | (sorted_contacts_df.variable == 'email3')))]
sorted_contacts_df.drop(columns='variable', inplace=True)

sorted_contacts_df['cleaned_phone'] = sorted_contacts_df.apply(lambda x: convert_phonenumber(x.cleaned_phone), axis=1)

In [23]:
odoo_leads = search_from_odoo(
    'crm.lead',
    [
        "&", "|", ["active", "=", True], ["active", "=", False], ["type", "=", "lead"]
    ],
    ['id', 'name', 'email_from', 'phone', 'create_date', 'type']
)
raw_odoo_leads_df = pd.DataFrame.from_dict(odoo_leads)
odoo_leads_df = raw_odoo_leads_df
odoo_leads_df.rename(columns={
    'email_from': 'cleaned_email'
}, inplace=True)
odoo_leads_df = odoo_leads_df.replace('', None)
odoo_leads_df = odoo_leads_df.replace(False, None)
odoo_leads_df['create_date'] = pd.to_datetime(odoo_leads_df['create_date'], errors='coerce')
odoo_leads_df['cleaned_phone'] = odoo_leads_df.apply(lambda x: convert_phonenumber(x.phone), axis=1)

In [25]:
selected_cw_df = cw_df[['id', 'name', 'cleaned_email', 'cleaned_phone', 'create_date']]
selected_odoo_contacts_df = sorted_contacts_df[['id', 'name', 'cleaned_email', 'cleaned_phone', 'create_date']]
selected_odoo_leads_df = odoo_leads_df[['id', 'name', 'cleaned_email', 'cleaned_phone', 'create_date']]

In [90]:
joined_contacts = pd.concat([selected_cw_df, selected_odoo_contacts_df, selected_odoo_leads_df])
joined_contacts = joined_contacts[(~joined_contacts.cleaned_email.isna()) | (~joined_contacts.cleaned_phone.isna())]
joined_contacts['cleaned_email'] = joined_contacts['cleaned_email'].str.lower()
joined_contacts = joined_contacts.sort_values(by='create_date', ascending=False)
joined_contacts = joined_contacts.drop_duplicates(subset=['cleaned_email', 'cleaned_phone'], keep='last')

In [140]:
df = joined_contacts
df['group_label'] = None

In [142]:
for index, row in df.iterrows():
    found_dup = df[(df.cleaned_email == row.cleaned_email)]
    if len(found_dup) > 1:
        label = found_dup.reset_index().at[0, 'id']
        duplicated_label('cleaned_email', row, 1, label)

In [143]:
for index, row in df.iterrows():
    found_dup = df[(df.cleaned_phone == row.cleaned_phone)]
    if len(found_dup) > 1:
        label = found_dup.reset_index().at[0, 'id']
        duplicated_label('cleaned_phone', row, 1, label)

In [144]:
duplicated_df = df[~df.group_label.isna()]
non_duplicated_df = df[df.group_label.isna()]
filtered_duplicated_df = duplicated_df.drop_duplicates(subset='group_label', keep='last')

In [155]:
date_range_2023 = filtered_duplicated_df[(filtered_duplicated_df['create_date'] >= '2023-10-01') & (filtered_duplicated_df['create_date'] <= '2023-12-31')]
filtered_duplicated_mql_2023 = date_range_2023.groupby([date_range_2023['create_date'].dt.date.rename('month')]).agg({'count'})['create_date']

In [156]:
date_range_2023 = non_duplicated_df[(non_duplicated_df['create_date'] >= '2023-10-01') & (non_duplicated_df['create_date'] <= '2023-12-31')]
non_duplicated_mql_2023 = date_range_2023.groupby([date_range_2023['create_date'].dt.date.rename('month')]).agg({'count'})['create_date']

In [157]:
non_duplicated_mql_2023

,count
month,
2023-10-02,53
2023-10-03,41
2023-10-04,53
2023-10-05,41
2023-10-06,46
2023-10-07,37
2023-10-08,1
2023-10-09,60
2023-10-10,32


In [158]:
filtered_duplicated_mql_2023

,count
month,
2023-10-01,4
2023-10-02,21
2023-10-03,30
2023-10-04,27
2023-10-05,37
2023-10-06,44
2023-10-07,21
2023-10-08,3
2023-10-09,25


In [159]:
result = non_duplicated_mql_2023.add(filtered_duplicated_mql_2023, fill_value=0)
result

,count
month,
2023-10-01,4.0
2023-10-02,74.0
2023-10-03,71.0
2023-10-04,80.0
2023-10-05,78.0
2023-10-06,90.0
2023-10-07,58.0
2023-10-08,4.0
2023-10-09,85.0


In [160]:
x = non_duplicated_df[(non_duplicated_df['create_date'] >= '2023-10-02') & (non_duplicated_df['create_date'] < '2023-10-03')].sort_values(by='create_date')

In [115]:
x[x.cleaned_phone == '+62 812-3147-107']

,id,name,cleaned_email,cleaned_phone,create_date,group_label
856,61265,Djohar Syafrinnul,None,+62 812-3147-107,2023-10-05 05:48:43,None


In [162]:
with pd.ExcelWriter("/Users/bcw-003/Downloads/test_2.xlsx") as writer:
    x.to_excel(writer, sheet_name='contacts', index=False)

In [165]:
xxx = filtered_duplicated_df[(filtered_duplicated_df['create_date'] >= '2023-10-02') & (filtered_duplicated_df['create_date'] < '2023-10-03')].sort_values(by='create_date')

In [166]:
with pd.ExcelWriter("/Users/bcw-003/Downloads/test_3.xlsx") as writer:
    xxx.to_excel(writer, sheet_name='contacts', index=False)

In [116]:
cw_df[cw_df.id == 61265]

,id,name,cleaned_email,phone,account_id,create_date,updated_at,additional_attributes,identifier,custom_attributes,last_activity_at,cleaned_phone


In [117]:
sorted_contacts_df[sorted_contacts_df.id == 61265]

,id,name,email,phone,create_date,cleaned_phone,cleaned_email
856,61265,Djohar Syafrinnul,None,+62 812-3147-107,2023-10-05 05:48:43,+62 812-3147-107,None


In [118]:
odoo_leads_df[odoo_leads_df.id == 61265]

,id,name,cleaned_email,phone,create_date,type,cleaned_phone


In [178]:
sorted_contacts_df[sorted_contacts_df.id == 632977]

,id,name,email,phone,create_date,cleaned_phone,cleaned_email


In [180]:
cw_df[cw_df.id == 632976]

,id,name,cleaned_email,phone,account_id,create_date,updated_at,additional_attributes,identifier,custom_attributes,last_activity_at,cleaned_phone
18853,632976,Ismed S,ismedst@gmail.com,+6285289404589,1,2023-10-02 01:48:37.728082,2023-10-02 01:56:21.513579,"{'city': 'Jaksel', 'country': '', 'description...",None,{},NaT,+62 852-8940-4589


In [177]:
odoo_leads_df[odoo_leads_df.id == 50348]

,id,name,cleaned_email,phone,create_date,type,cleaned_phone
2535,50348,Ismed S,ismedst@gmail.com,+62 851-7306-0800,2023-10-02 01:48:45,lead,+62 851-7306-0800


In [169]:
df[df.cleaned_phone == '+62 851-7306-0800']

,id,name,cleaned_email,cleaned_phone,create_date,group_label
18854,632977,Ismed S,None,+62 851-7306-0800,2023-10-02 01:48:56.665165,632977
2535,50348,Ismed S,ismedst@gmail.com,+62 851-7306-0800,2023-10-02 01:48:45.000000,632977


In [173]:
duplicated_df[duplicated_df.group_label == 632977]

,id,name,cleaned_email,cleaned_phone,create_date,group_label
18854,632977,Ismed S,None,+62 851-7306-0800,2023-10-02 01:48:56.665165,632977
2535,50348,Ismed S,ismedst@gmail.com,+62 851-7306-0800,2023-10-02 01:48:45.000000,632977
18853,632976,Ismed S,ismedst@gmail.com,+62 852-8940-4589,2023-10-02 01:48:37.728082,632977


In [172]:
non_duplicated_df[non_duplicated_df.cleaned_phone == '+62 851-7306-0800']

,id,name,cleaned_email,cleaned_phone,create_date,group_label


In [174]:
filtered_duplicated_df[filtered_duplicated_df.group_label == 632977]

,id,name,cleaned_email,cleaned_phone,create_date,group_label
18853,632976,Ismed S,ismedst@gmail.com,+62 852-8940-4589,2023-10-02 01:48:37.728082,632977


In [124]:
coba_1 = df[df.cleaned_phone == '+62 822-3715-9990']

In [126]:
temp_df = df

In [128]:
df = coba_1

In [138]:
for index, row in df.iterrows():
    found_dup = df[(df.cleaned_phone == row.cleaned_phone)]
    if len(found_dup) > 1:
        label = found_dup.reset_index().at[0, 'id']
        duplicated_label('cleaned_phone', row, 1, label)

In [139]:
df

,id,name,cleaned_email,cleaned_phone,create_date,group_label
2297,50670,Abigail Angelia Hanas,abigailhanas28@gmail.com,+62 822-3715-9990,2023-10-05 05:48:55.000000,50670
19152,633275,Abigail Angelia Hanas,None,+62 822-3715-9990,2023-10-06 01:26:15.405665,50670
